In [1]:
from selenium import webdriver
driver=webdriver.Firefox()
driver.implicitly_wait(3)
driver.get("https://www.fundsexplorer.com.br/ranking")

In [2]:
from selenium.webdriver.common.by import By
fecha_aceitar = '//*[@id="hs-eu-confirmation-button"]'#"/html/body/div[2]/div/div[2]/div/a"
driver.find_element(By.XPATH,fecha_aceitar).click()

In [3]:
#-------------# Muda o contexto para o iframe 
iframe_element = driver.find_element(By.XPATH, "//iframe[@title='Popup CTA']")
driver.switch_to.frame(iframe_element)
# Encontra o botão "X" dentro do iframe
close_button = driver.find_element(By.XPATH, "/html/body/div/div[1]")
# Clique no botão "X"
close_button.click()
# Volte para o contexto principal
driver.switch_to.default_content()
#------------------------------------


In [4]:
# Scroll up the window by a specific number of pixels
# For example, scroll up by 200 pixels
scroll_distance = 500
driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

In [5]:
driver.find_element(By.XPATH,'//*[@id="colunas-ranking__select-button"]').click()
driver.find_element(By.XPATH,'/html/body/div[7]/div[1]/div/div[2]/div[2]/ul/li[1]/label').click()

In [ ]:
from io import StringIO
import pandas as pd

html_str = driver.page_source
tabelas_html = pd.read_html(StringIO(html_str))

df=tabelas_html[0]

In [ ]:
# dados = []
# dadosTabela = driver.find_element(By.XPATH,'//div/table[contains(@class,"default-fiis-table__container__table")]')

# #print(dadosTabela.text)  

# for linha in dadosTabela.find_elements(By.TAG_NAME,"tr") :
#     linhaDados = []
#     for coluna in linha.find_elements(By.TAG_NAME,"td"):
#         # print(coluna)
#         linhaDados.append(coluna.text)
#     dados.append(linhaDados)


# import pandas as pd
# colunas = ['Fundos','Setor','Preço Atual (R$)','Liquidez Diária (R$)',#
#     'P/VP','Último Dividendo','Dividend Yield','DY (3M) Acumulado',#
#     'DY (6M) Acumulado','DY (12M) Acumulado','DY (3M) média','DY (6M) média',#
#     'DY (12M) média','DY Ano','Variação Preço','Rentab. Período','Rentab. Acumulada',#
#     'Patrimônio Líquido','VPA','P/VPA','DY Patrimonial','Variação Patrimonial',#
#     'Rentab. Patr. Período','Rentab. Patr. Acumulada','Quant. Ativos' ,'Volatilidade' , #
#     'Num. Cotistas' , 'Tax. Gestão' , 'Tax. Performance' , 'Tax. Administração']

# df = pd.DataFrame(dados,columns=colunas)



In [13]:
import os
data_path = str(os.getcwd()) + r"/data/"

In [14]:
df.to_csv( data_path+"FE.csv" , sep=";" , decimal="," )

In [15]:
driver.close()